In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
import numpy as np
import pandas as pd
from sklearn.cluster import KMeans
from kneed import KneeLocator
from sklearn.metrics.pairwise import euclidean_distances
from sklearn.preprocessing import StandardScaler, normalize
from sklearn.linear_model import LinearRegression
from sklearn import model_selection
import matplotlib.pyplot as plt
import psycopg2
import seaborn as sns
import pickle
import os
import sys

In [4]:
#Ignoring Warnings

import warnings
warnings.filterwarnings('ignore')

In [5]:
sys.path.append(os.path.abspath(os.path.join('../scripts')))
from preprocessing import PreProcess
from overview import Overview
from plot import Plot
from logger import Logger

In [6]:
# Initialize logger
logger = Logger("userSatisfaction.log").get_app_logger()
logger.info("Initialized user satisfaction logger")

In [7]:
#reading the cleaned data in EDA
df_org = pd.read_csv("C:\\Users\\1221\\Desktop\\Telecommunication-Data\\data\\cleaned telecom data.csv")
df = df_org.copy()

Satisfaction Analysis

In [8]:
# Import user experience data
user_exp = pd.read_csv('../data/user_exp.csv')

# Import user engagement data
user_eng = pd.read_csv('../data/user_eng.csv')

In [9]:
user_exp.head()

,MSISDN/Number,cluster,avg_rtt_total,avg_tp_total,total_avg_tcp_total
0,-0.002692,1,0.024925,0.055505,0.103643
1,-0.002692,1,0.024834,0.055524,0.103643
2,-0.002692,1,0.026789,0.055522,0.103643
3,-0.002692,1,0.026209,0.055555,0.103153
4,-0.002692,0,0.023633,0.081818,0.103244


In [10]:
user_eng.head()

,MSISDN/Number,cluster,sessions,Dur. (ms).1,total_data
0,-0.002692,2,1,0.146799,1.139608
1,-0.002692,1,1,0.944041,-1.812243
2,-0.002692,0,1,0.372328,0.256948
3,-0.002692,1,1,-0.679254,-2.059544
4,-0.002692,2,2,-2.132773,2.926302


In [11]:
user_eng_df = user_eng.set_index('MSISDN/Number')[
    ['sessions','Dur. (ms).1', 'total_data']]
scaler = StandardScaler()
scaled_array = scaler.fit_transform(user_eng_df)
pd.DataFrame(scaled_array).head(5)

,0,1,2
0,-0.232540,0.076622,0.680985
1,-0.232540,0.492745,-1.082925
2,-0.232540,0.194338,0.153542
3,-0.232540,-0.354539,-1.230702
4,0.359739,-1.113207,1.748642


In [12]:
data_normalized = normalize(scaled_array)
pd.DataFrame(data_normalized).head(5)

,0,1,2
0,-0.321338,0.105881,0.941027
1,-0.191822,0.406465,-0.893303
2,-0.684482,0.572034,0.451952
3,-0.178644,-0.272368,-0.945464
4,0.170987,-0.529116,0.831143


Calculate Engagement and Experience Scores